# **Two-Stage-Retrieval LLM RAG**


## **Table of Contents**


* [1. Introduction & Key Features](#1-introduction--key-features)
* [2. Project Structure](#2-project-structure)
    * [2.1 Input Data](#21-input-data)
    * [2.2 Source Code](#22-source-code)
    * [2.3 Output Data](#23-output-data)
* [3. Workflow Summary](#3-workflow-summary)
* [4. Demonstration](#4-demonstration)
    * [4.1 Data Acquisition and Preparation](#41-data-acquisition-and-preparation)
    * [4.2 Methodology and Pipeline Design](#42-methodology-and-pipeline-design)
    * [4.3 Execution and Results](#43-execution-and-results)
* [5. Data Analysis and Inference](#5-data-analysis-and-inference)
    * [5.1. Defining Metrics and Data Collection](#51-defining-metrics-and-data-collection)
    * [5.2. The Evaluation Challenge: Defining "Likeness"](#52-the-evaluation-challenge-defining-likeness)
    * [5.3. Pipeline Performance and Inference](#53-pipeline-performance-and-inference)
    * [5.4. Conclusion: Optimal Pipeline and Conditions](#54-conclusion-optimal-pipeline-and-conditions)
* [6. Authors](#6-authors)
* [7. License](#7-license)

# **1. Introduction & Key Features**


This project implements a question-answering (Q&A) system over a technical research paper from Meta SuperIntelligence Labs. The goal is to compare how different retrieval-augmented generation (RAG) setups help a large language model (LLM) answer multiple-choice questions.

The following retrieval pipelines are compared:

- **LLM Baseline** – Direct generation without retrieval.

- **BM25** – Classic sparse keyword search (TF-IDF based).

- **Dense Retrieval** – Embedding-based semantic retrieval.

- **(Bonus) Hybrid Retrieval** – Combination of BM25 and Dense Retrieval.

- **(Bonus) Hybrid Retrieval + Cross-Encoder** – A SOTA approach where a high-recall retriever fetches candidates (Hybrid, k=20) and a Cross-Encoder reranks them for high precision (k=5).


To ensure robustness, reproducibility, and scientific rigor, this project integrates several software engineering patterns:

- **Singleton Pattern for Retrieval Engine:** The `RetrievalEngine` class uses a Singleton pattern with **Lazy Loading**. This prevents memory overhead by loading heavy models only when necessary and ensures consistent database connections across the pipeline.

- **Smart Ingestion Strategy:** We compare **Recursive Chunking** (optimized with high overlap to preserve scientific context) against **Semantic Chunking** (experimental), which uses embedding distances to segment text by topic.

- **Automated "Judge" Evaluation:** Instead of relying solely on the final answer accuracy, we implement a **Ground Truth Verification** system. It compares the retrieved chunks against the source reference in the dataset to detect lucky guesses.

- **Two-Stage Architecture:** A funnel approach that prioritizes *Recall* in the first stage (Hybrid Search) and *Precision* in the second stage (Cross-Encoder Re-ranking).

# **2. Project Structure**

 The project is organized into multiple files under the src/ folder, with a clear separation of responsibilities. Here’s how it works:

## 2.1 **Input Data**

  All of the following are on the `data/` directory:
- `data/questions.json` - Contains 70 multiple-choice questions extracted from a technical research paper.
Each question includes: the correct answer, three distractors, and an optional reference to the source paper.

- `data/chroma_db` - Embedded vector data base.
- `data/enunciado.pdf` - The problem statement already described.
- `data/paper_refrag.pdf` - The technical paper from which questions and answers are extracted.


## 2.2 **Source Code**
- `main.py` - Entry point of the project.
Supports *Local* mode (`results/local_results/`) and *Persistent* mode (`results/persistent_results/<test_name>/`).  
    - Calls the pipeline, saves final results, and generates plots.
  

  All of the following are on the `src/` directory:
- `launcher.py` - Sets up the environment for experiments.
    - Initializes the vector database (ChromaDB).
    - Clears previous results if needed.
    - Creates all necessary directories (plots, final CSVs, etc.).

- `src/ingestion.py` - Prepares and creates the data base. Invoked by the launcher.

- `queries.py` - Main logic to execute questions.
    - For each question and method it:
        - Sends the query to the LLM and retrieves documents.
        - Computes accuracy and evidence verification.
        - Stores partial results in results/resultados_parciales.csv.
        - Returns a DataFrame with all results for further evaluation.

- `rag_pipeline.py` - Implements RAG logic for different retrieval methods
    - Contains functions to verify ground truth against retrieved documents.
     Computes retrieval scores and status tags for each answer.

- `retrieval.py` - Implements the retrieval engine.
    - Provides a singleton engine to handle different retrieval methods efficiently.

- `evaluation.py` - Evaluates the results and generates dashboards.
    - Accuracy per method  
    - RAG quality distribution  
    - Response latency  
    - Retrieval fidelity

## 2.3 **Output Data**

    Found on the `results/` directory:

- **Partial results** - Always stored in `results/resultados_parciales.csv`.
    - Updated after each question is processed.

- **Final results** - Stored in `results/local_results/` or `results/persistent_results/<test_name>`.
    - File name: `resultados_finales.csv`.

- **Plots/Dashboard** - Stored in `plots/` inside the corresponding results folder.
    - Include:
        - Bar charts for accuracy and RAG quality
        - Boxplots for response latency
        - Violin plots for retrieval fidelity

> **Tip:** Run `main.py try_n` on the terminal in orden to save the try number n results in that directory<br> in orden not to overwrite other results.

# **3. Workflow Summary**

1. Load questions dataset (questions.json).

2. Initialize vector database (ChromaDB).

3. For each question:
    - Retrieve relevant documents (BM25 / Dense / Hybrid / Hybrid + Cross-Encoder).
    - Query LLM for an answer.
    - Verify against ground truth.
    - Save partial results.

4. After all questions are processed:
    - Concatenate all results.
    - Save final results CSV.
    - Generate plots/dashboard for comparison and analysis.

# **4. Demonstration**
This section initiates the transition from the theoretical design to the **operational demonstration** of our Two-Stage Retrieval system. We will trace the execution flow, relying on the **modular architecture** established within the `src/` directory.

For the stability and reproducibility of the results, it is of vital importance that the project operates within a dedicated virtual environment (Anaconda/Miniconda). All complex dependencies (LangChain, ChromaDB, Sentence-Transformers) must be installed prior to execution by running: `pip install -r requirements.txt`. In addition, a valid API key for Google AI Studio must be provided in a private file called `.env`. The content of this file should be similar to: `GOOGLE_API_KEY="AI..."`

The initial code block below serves two primary engineering purposes:
1.  **Dependency Loading:** Loading the necessary core libraries (`pandas`, `matplotlib`) and setting up environment variables (`.env`).
2.  **Path Resolution:** Establishing the correct pathing (`sys.path.append`) to ensure all subsequent modules from our **clean `src/` architecture** are correctly imported and ready for use.

In [ ]:
# --- IMPORTS Y SETUP ---
import sys
import os
import time
import re
import json
import pandas as pd
import matplotlib.pyplot as plt
from src.evaluation import plot_accuracy, plot_rag_quality, plot_latency, plot_retrieval_score, setup_plot_style
from src.rag_pipeline import verify_ground_truth_v1, query_rag
from dotenv import load_dotenv

# Añadimos la raíz del proyecto al path para poder importar 'src'
sys.path.append(os.path.abspath("."))

# Load environment variables
load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")

if not API_KEY:
    print("Error: GOOGLE_API_KEY no está configurada en las variables de entorno.")
    sys.exit(1)

print("Variable de entorno y librerías cargadas correctamente.")

Variable de entorno cargada correctamente.


## 4.1 Data Acquisition and Preparation

The initial stage of the RAG pipeline focuses on **Data Pre-processing**. This involves breaking down the source document (`paper_refrag.pdf`) into discrete units of knowledge (**chunks**) that are stored in **ChromaDB**.

### Chunking Methodologies

We implemented two primary chunking strategies, selectable via the `CHUNKING_METHOD` variable in `src/ingestion.py`:

1.  **Recursive Chunking (Recursive Character Splitter):** This is the baseline method, optimized for technical context. It relies on rules of fixed length (e.g., **1200 characters** with **350 characters of overlap**).

2.  **Semantic Chunking:** The advanced method. It utilizes an **embedding model** to compute the similarity between consecutive sentences, identifying natural **topic shifts** to define chunk boundaries.

### Why Semantic Chunking?

We chose to focus on **Semantic Chunking** because it addresses the primary weakness of RAG in technical domains: **avoiding false negatives** in retrieval.

In dense scientific papers, naive splitting often cuts conclusions, cross-references, or complex sentences in half. This leads to:

* **Loss of Context:** The embedding model creates an inaccurate vector because the sentence is fragmented.

* **Lucky guesses:** The system answers correctly, but the strict evaluation judge fails to find the complete reference phrase because it was broken up.

By using **Semantic Chunking**, we ensure that each unit of information passed to the retriever has the **highest possible semantic coherence**, which directly improves the **Retrieval Recall** and **Fidelity** metrics in our evaluation dashboard.



---

Once the text is chunked, the system proceeds to **initialize the Retriever** in a **Lazy Loading** state, ensuring the database connection is established efficiently right before the first query is made.

The code below creates a new database if neccesary and initializes our retrieval engine for the next steps. To test the difference in runtime between recursive and semantic chunking set `rebuild_db=True` Note: due to our implementation, changing the chunking method requires to restart the kernel.

In [ ]:
# --- GESTIÓN DE DATOS Y VERIFICACIÓN ---
import os
from src.ingestion import db_setup
from src.retrieval import RetrievalEngine

try:
    # 1. GARANTÍA DE EXISTENCIA DE BD (Ingesta)
    db_setup(rebuild_db=False, chunking_method="recursive") # Opciones: "recursive", "semantic"
    
    # 2. CONEXIÓN AL MOTOR (Singleton)
    print("\n⚙️ Conectando al motor de recuperación...")
    engine = RetrievalEngine.get_instance()
    # 3. PRUEBA DE RECUPERADOR
    # Esto asegura que ChromaDB y los Embeddings están cargados en RAM
    test_retriever = engine.get_retriever("hybrid", k=1) # Así aseguramos de que se inicien dense y bm25
    
    # Obtenemos estadísticas reales de la BD
    if engine.db:
        doc_count = engine.db._collection.count()
        print(f"✅ SISTEMA OPERATIVO")
        print(f"   - Estado: Base de Datos Conectada")
        print(f"   - Ruta: ./data/chroma_db")
        print(f"   - Chunks Indexados: {doc_count}")
        print(f"   - Modelo Embeddings: all-MiniLM-L6-v2")
    
except Exception as e:
    engine.unload_db()  # Liberamos memoria antes de la prueba
    print(f"\n❌ ERROR CRÍTICO DE INICIALIZACIÓN: {e}")
    print("   - Verifica que la base de datos esté construida correctamente.")


🔌 Desconectando motor de búsqueda...
📄 Cargando PDF...
   -> PDF cargado: 30 páginas.
✂️ Procesando fragmentos
   -> Generados 128 fragmentos.
🧠 Guardando vectores en disco...
💾 Base de datos guardada exitosamente.
✅ Setup completado.

⚙️ Conectando al motor de recuperación...
✅ SISTEMA OPERATIVO
   - Estado: Base de Datos Conectada
   - Ruta: ./data/chroma_db
   - Chunks Indexados: 128
   - Modelo Embeddings: all-MiniLM-L6-v2


## 4.2 Methodology and Pipeline Design

Our solution employs a modular two-stage pipeline designed to overcome the inherent limitations of standard vector search. The process is defined by two primary phases: Retrieval (Sourcing the evidence) and Evaluation (Verifying the source).

### The Execution Flow

**1. Retrieval Phase (Engine)**: The system first receives the query and selects the retrieval strategy (BM25, Dense, Hybrid, or Cross-Encoder). 

**añadir que hace cada bm, dense e hybrid**

If the advanced Cross-Encoder method is selected, the system executes a Two-Stage Retrieval funnel:
- Stage 1 (Broad Search): The Hybrid Retriever fetches a wide pool of candidates (e.g., $k=20$).
- Stage 2 (Fine Filter): The Cross-Encoder reads all 25 candidates and re-ranks them, selecting only the top $k=5$ highest-precision documents for the LLM.

**2. Generation Phase (LLM)**: The final context and the strict prompt are sent to Gemini 2.5 Flash Lite for answer generation.

**3. Verification Phase (Judge)**: The system immediately computes two critical metrics:

- Accuracy Check: Compares the predicted letter (A/B/C/D) against the ground truth answer key.

- Fidelity Check: Compares the retrieved documents against the specific reference text from the paper (paper_reference) using a Fuzzy Matching Judge or the LLM-as-a-Judge. This determines the Status Tag (e.g., Perfect RAG vs. Lucky Guess).

**4. Logging**: All metrics (Time, Score, Status) are logged to the final CSV file.

---
The following code shows an example of steps 2 and 3 with one question. Note that this demo question can be changed [0-69] and that the run may seem slow due to an intended sleep time to avoid API rate limits problems.


In [6]:
# Aseguramos que la pregunta de demo esté cargada (asumimos que questions[0] es la pregunta de prueba)
path_json = "./data/questions.json"
if not os.path.exists(path_json):
    print("❌ ERROR: No encuentro 'data/questions.json'")
with open(path_json, "r", encoding="utf-8") as f:
    questions = json.load(f)

if questions:
    q = questions[0] # Change index for different questions
    methods = ["baseline", "bm25", "dense", "hybrid", "cross_encoder"]

    print(f"PREGUNTA: {q['question']}")
    
    for method in methods:
        print(f"   [{method.upper()}] Procesando...", end=" ")

        start_time = time.time()
        
        # query_rag devuelve (raw_answer, retrieved_docs)
        raw_answer, retrieved_docs = query_rag(
            question=q['question'],
            options=q['answers'],
            method=method,
            api_key=API_KEY
        )
        
        end_time = time.time()
        latency = end_time - start_time
        time.sleep(2)  # Pequeña pausa para no saturar API

        # Limpieza de respuesta
        match = re.search(r'(?i)\b([A-D])\b', raw_answer)
        predicted_letter = match.group(1).upper() if match else "X"

        # Evaluación básica
        correct_letter = q['correct_answer']
        is_correct = (predicted_letter == correct_letter)

        # 3. --- NUEVO: JUEZ DE GROUND TRUTH + LLM Judge ---
        paper_ref = q.get('paper_reference', "")
        found_evidence, evidence_score = False, 0.0

        # --- SIMULACIÓN DE LA EVALUACIÓN ---
        # Usamos la función de verificación para obtener el score real (necesario para el status)
        found_evidence, retrieval_score = verify_ground_truth_v1(retrieved_docs, paper_ref)
        
        # Lógica del Status (Simplificada)
        is_correct = (predicted_letter == correct_letter)
        
        if is_correct and found_evidence:
            status_tag = "✅ ACIERTO PERFECTO (RAG)"
        elif is_correct and not found_evidence:
            status_tag = "⚠️ ACIERTO SUERTE"
        else:
            status_tag = "❌ INCORRECTO"

        print(f"{status_tag} (Pred: {predicted_letter} | T: {latency:.2f}s)")

PREGUNTA: What is the primary mechanism through which the REFRAG framework achieves a reduction in computational complexity for attention?
   [BASELINE] Procesando... ⚠️ ACIERTO SUERTE (Pred: C | T: 0.61s)
   [BM25] Procesando... ⚠️ ACIERTO SUERTE (Pred: C | T: 0.50s)
   [DENSE] Procesando... ✅ ACIERTO PERFECTO (RAG) (Pred: C | T: 0.50s)
   [HYBRID] Procesando... ✅ ACIERTO PERFECTO (RAG) (Pred: C | T: 0.60s)
   [CROSS_ENCODER] Procesando... ✅ ACIERTO PERFECTO (RAG) (Pred: C | T: 2.20s)


## 4.3 Execution and Results

In this final phase, we execute the full benchmark to evaluate the performance of each retrieval strategy.

Experiment Setup:

**- Dataset**: 70 multiple-choice questions derived from the REFRAG paper.

**- Methods Evaluated**: BM25, Dense, Hybrid, and Cross-Encoder.

**- Metrics**: Accuracy, Latency, Retrieval Fidelity, and RAG Quality Diagnosis.

**Note**: Running the full experiment takes approximately 15-20 minutes due to the computational cost of the Cross-Encoder and the API rate limits. For the purpose of this notebook, we will load the pre-computed results.

In [ ]:
RESULTS_PATH = "./results/demo_results.csv"
# 1. CARGA DE DATOS (Real vs. Demo)
if os.path.exists(RESULTS_PATH):
    print(f"📂 Cargando resultados reales desde: {RESULTS_PATH}")
    df_results = pd.read_csv(RESULTS_PATH)
else:
    print("Error al cargar resultados reales.")

# 2. GENERACIÓN DE DASHBOARD VISUAL
print("\n📊 Generando Dashboard de Rendimiento...")
setup_plot_style()

# Creamos una figura grande para mostrar todo junto (Opcional, o llamamos a las funciones una a una)
# Aquí llamamos a tus funciones de evaluation.py que guardan los PNGs y los mostramos.

# Gráfica 1: Precisión
plot_accuracy(df_results)
plt.show()

# Gráfica 2: Calidad RAG (El gráfico de colores)
plot_rag_quality(df_results)
plt.show()

# Gráfica 3: Latencia
plot_latency(df_results)
plt.show()

# Gráfica 4: Fidelidad
plot_retrieval_score(df_results)
plt.show()

print("✅ Análisis completado.")

## Key Findings
Based on the visualized data, we can draw the following conclusions:

**Precision vs. Speed Trade-off**: The Cross-Encoder pipeline demonstrates superior accuracy and retrieval fidelity, effectively filtering out noise. However, this comes at the cost of higher latency (~3s vs 1s), confirming our hypothesis that Two-Stage Retrieval is ideal for offline or high-stakes QA, but Hybrid Retrieval is better for real-time applications.

**The "Luck" Factor**: The RAG Quality plot reveals that a significant portion of "correct" answers in baseline methods (BM25/Dense) are classified as "Lucky Guesses" (Yellow). The Cross-Encoder significantly converts these into "Perfect RAG" (Green) by ensuring the retrieved context actually contains the semantic answer.

**Overall Winner**: The Hybrid + Cross-Encoder architecture provides the most robust performance for scientific document analysis, minimizing hallucinations and reasoning failures.

# **5. Data Analysis and Inference**


Our project automatically generates four plots associated with the metrics we use:

1. Accuracy on answer selected
2. RAG quality
3. Latency
4. Retrieval fidelity



## 5.1. Defining Metrics and Data Collection

The evaluation system in the src/evaluation.py module processes the raw data from queries.py to produce four key metrics for an integral performance assessment. This table details how data is collected and valued for each metric within the project:

| Metric | Data Collection Method (Source Module) | Evaluation (Scale/Type) |
| :--- | :--- | :--- |
| **1. Accuracy on answer selected** | Comparison of the LLM's final chosen option against the `correct_answer` field in `questions.json`. | Binary (0 or 1). |
| **2. RAG Quality** | Result of the **"Judge"** comparing the final answer to the retrieved context and ground truth. | Categorical (Correct, Lucky Answer, Reasoning Fail, Incorrect). |
| **3. Latency** | Time for the retrieval and generation steps combined. | Continuous (s). |
| **4. Retrieval fidelity** | Measures the relevance of the retrieved documents against the `source_reference` (ground truth chunk location). |  Ranking Metric (based on the position of the relevant chunk). |


However, it is important not to ignore how the **"Judge"** is made for the RAG Quality, nor the function that evaluates the retrieval fidelity: 


### RAG Quality Judges

The **RAG Quality** metric requires a **Judge** to determine whether the LLM's answer is *justified* by the retrieved context or if it was a **"Lucky Answer"**. We have explored several methodologies, implemented in the `src.evaluation.py` module through the functions `verify_ground_truth_v1`, `v2`, and `v3`:

1.  **Judge V1 (Simple/Fuzzy Text Matching):** This uses substring search and the *Fuzzy Matching* algorithm (`SequenceMatcher` from `difflib`) in `verify_ground_truth_v1` and `verify_ground_truth_v3`. This approach is considered **conservative** and **literal**, requiring a match of $\ge 50\%$ of the ground truth text (`ground_truth_ref`) within the retrieved context. If it fails, the LLM is marked as "hallucinating" (Lucky  when the LLM got the right answer but without a correct finding; or a Reasoning Fail when it got the right finding but the wrong answer).

2.  **Judge V2/V3 (LLM Judge):** This employs an LLM (`verify_ground_truth_v2` and `verify_ground_truth_v3`) to determine justification.
    * **V2 (Literal-Tolerant):** Based on `JUDGE_PROMPT2`, it instructs the LLM to ignore only formatting artifacts (hyphens, line breaks) but demands a **literal match** of the word sequence.
    * **V3 (Semantic):** Based on `JUDGE_PROMPT`, it instructs the LLM to look for **semantic matching** (same meaning) even if the words are different. We have implemented a two-stage cascading check judge. Judge V1 acts as a fast, low-cost filter, handling all deterministic verifications . Only when V1 fails to issue a positive verdict does the flow proceed use a LLM as the final contextual judge. This approach prioritizes efficiency with V1 while ensuring accuracy and nuanced evaluation with the LLM.

### Retrieval fidelity

The **Retrieval Fidelity** metric evaluates the quality of the search engine. It is calculated by measuring whether the text fragment containing the correct answer (the `ground_truth_ref` or reference *chunk*) was retrieved.

The validation of whether the reference *chunk* was retrieved is performed using the `verify_ground_truth_v1` function or its variants. Once its presence is validated, the Retrieval Fidelity metric is defined as:

$$\text{Fidelity} = \frac{\text{Number of the letters in a sequence of the retrieved text that are exactly the same as the expected answer}}{\text{Total number of letters in the expected answer}}$$

This metric is **fundamental** because it acts as an upper bound for **RAG Quality**: if half of the absolute truth *chunk* is not retrieved, it is impossible for the LLM to answer justifiably (except by internal knowledge in the `baseline` case).


## 5.2. The Evaluation Challenge: Defining "Likeness"
Therefore, we will discuss each **pipeline's performance** in each metric described, but before that, we must take into account the **methodology** considered for the evaluation of the RAG quality and the retrieval fidelity judge. This is due to the fact that these metrics are attached to how we evaluate likeness and the threshold of a "lucky answer." We will give an insight into this idea:

- The plots in `results/persistent_results/27-11-1/plots/` are obtained through a judge that is based on text matching:


<center>
    <img src="results/persistent_results/27-11-1/plots/2_rag_quality_pct.png" alt="Text Matching Judge (Conservative)" width="45%"/>
</center>

- Whereas the ones in `results/persistent_results/8-11-1/plots/` have an **LLM judge** that searches for the semantic match: 


<center>
    <img src="results/persistent_results/28-11-1/plots/2_rag_quality_pct.png" alt="LLM Judge (Semantic)" width="45%"/>
</center>


Suddenly, all the pipelines have improved. However, when **manually supervised**, neither of the judges has a **flawless performance** at this task. The first judge **incorrectly** marks as "lucky answers" a group that is, in fact, a "correct answer," and the second does the opposite.

This **false positives** and **false negatives** concept is an extremely important idea in statistics and in other science fields, and the general conclusion for this problem is to be **conservative**.

For this reason, we will mention other methodologies we have developed but only show and explore the ones with a risk-averse approach.

For this reason, we have chosen the function **`verify_ground_truth_v1` (Judge V1 - Text/Fuzzy Matching)** to calculate the **RAG Quality** and **Retrieval Fidelity** metrics that will be presented next. This approach minimizes the possibility of **false positives**, ensuring the LLM is only marked as "justified" when there is clear, strict text matching evidence in the retrieved context.


## 5.3. Pipeline Performance and Inference

explicar como de bueno es cada pipeline en cada metrica y decir cual es mejor y en que condiciones

## 5.4. Conclusion: Optimal Pipeline and Conditions
The analysis of the four key metrics—**Accuracy**, **RAG Quality (Justification)**, **Latency**, and **Retrieval Fidelity**—allows us to establish the most suitable Retrieval-Augmented Generation (RAG) pipeline for this specific academic validation use case, balancing performance and cost.

### 🎯 The Necessity of RAG: The Control Group

The **Baseline** method (no context retrieval) acts as our control group:
* It achieves a low **Accuracy** of **65.7%** .
* Its **RAG Quality (Justification)** is **0.0%**.

The fact that all active RAG methods immediately jump to a maximum **Accuracy** of **98.6%** confirms that **RAG is essential** for solving the questions in the corpus. The analysis focuses, therefore, on **which pipeline offers the best justification and latency** above this high accuracy floor.

---

### 📈 Hierarchical Progression and Performance

Performance is analyzed in three stages to understand the marginal contribution of each technique:

#### 1. Simple Engines (BM25 vs. Dense)
Comparing the simple engines reveals the importance of retrieval method quality:

* The **BM25** method (keyword search) achieves maximum **Accuracy** (**98.6%**) and a **RAG Quality** of **34%**. Its **Latency** is the lowest among active RAG methods (median of **0.70s**).
* The **Dense** method (semantic vector search) has lower **Accuracy** (**90.0%**) and a slightly higher **RAG Quality** (**39%**), with similar latency.

**Intermediate Conclusion:** For this type of technical corpus, the **BM25** method provides a more reliable and faster initial retrieval than the simple **Dense** vector search.

#### 2. From Simple to Hybrid: The Justification Jump 
The **Hybrid** method combines the robustness of **BM25** with the semantic capability of **Dense**, achieving a drastic increase in justification:

* **Improvement:** Moving from **BM25 (34%)** to **Hybrid (64%)**, the percentage of **Perfect Justification** nearly **doubles**. This occurs with only a marginal latency penalty, as the median only increases from 0.70s to 0.80s .

**Intermediate Conclusion:** The **Hybrid Pipeline** provides the best performance foundation, achieving the highest justification with very low latency.

#### 3. From Hybrid to Reranking (Cross-Encoder): The Cost of Refinement
The **Cross-Encoder** method adds a costly re-ranking step to the Hybrid process:

* **Quality vs. Latency:** The **Cross-Encoder RAG Quality** is **61%** , falling slightly *below* the simple **Hybrid** method (**64%**). However, the **Latency cost** is severe, increasing from **0.80s** (Hybrid) to **1.70s** (Cross-Encoder) .

**Final Conclusion:** In this use case, re-ranking does not provide a quality improvement that justifies the latency penalty of over **100%**.

---

### Summary: The Optimal Pipeline

The **Hybrid Pipeline** is the **optimal method** as it offers the best balance between Justification and Latency.

#### Comprehensive Performance Table

| Pipeline | Accuracy (%) | RAG Quality (Perfect Match) (%) | Latency (Median, s) |
| :--- | :--- | :--- | :--- | :--- |
| **Baseline** | 65.7% | 0.0% | 0.55 |
| **BM25** | **98.6%** | 34% | **0.70** |
| **Dense** | 90.0% | 39% | 0.85 |
| **Hybrid** | **98.6%** | **64%** | 0.80 |
| **Cross-Encoder** | **98.6%** | 61% | 1.70 |

### Conditions for Selection

The choice of the final pipeline must be based on the application's priority:

* **Maximum Efficiency (Speed + Quality):** The **Hybrid Pipeline** is the clear choice. It offers the highest justification rate with low latency. It is the most suitable for RAG systems in production.
* **Absolute Priority to Justification (Ignoring Cost):** Although **Hybrid** is marginally superior, **Cross-Encoder** could be considered if the goal is to demonstrate the technical viability of the most advanced justification technique.
* **Simple/Constrained Environment:** The **BM25** method remains a very strong option, offering maximum accuracy and low latency if infrastructure constraints prevent the use of vectors.

# **6. Authors**
- Jorge Barbero Morán – UCM, Faculty of Mathematics
- David Marcos Jimeno – UCM, Faculty of Mathematics


# **7. License**
This project is licensed under the **MIT License**